## Data augmentation

Our dataset is very badly equilibrated. The goal is to augmente the data with have in a few less-representated grades.

In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("..//datasets//avisassurance_train.csv", encoding="utf-8", sep = ";")
train

,date,note,auteur,avis,assureur,produit
0,06 septem...,5,brahim--k-131532,"Meilleurs assurances, prix, solutions, écoute,...",Direct Assurance,auto
1,03 mai 20...,4,bernard-g-112497,"je suis globalement satisfait , sauf que vous ...",Direct Assurance,auto
2,21 mars 2...,5,virginie-t-107352,Prix tres abordable plusieurs options s'offren...,Direct Assurance,auto
3,10 juin 2...,4,boulain-f-116580,"je satisfait du service, une réponse très rapi...",L'olivier Assurance,auto
4,29 janvie...,1,ouaille31-51798,"Client depuis plus de 25 ans, très déçu de cet...",Matmut,auto
...,...,...,...,...,...,...
24100,22 mars 2...,1,hophop-107522,Assurance moto chez la mutuel des motards en F...,Mutuelle des Motards,moto
24101,06 décemb...,1,tzl-81680,Même les demandes les plus simples n'aboutisse...,Allianz,habitation
24102,14 avril ...,1,jmr-72500-110395,"En décembre 2019, j'ai souscrit à un contrat C...",Cegema Assurances,sante
24103,11 juille...,3,cris-77532,Je suis assurer à la gmf depuis plus de 15 ans...,GMF,auto


In [3]:
import gensim.downloader as api

In [13]:
twitter_25_model = api.load('word2vec-google-news-300')  
twitter_25_model.most_similar('awesome', topn=5)

[==================================================] 100.0% 1662.8/1662.8MB downloaded


[('amazing', 0.8282865285873413),
 ('unbelievable', 0.7464959621429443),
 ('fantastic', 0.7453290224075317),
 ('incredible', 0.7390913963317871),
 ('unbelieveable', 0.6678115725517273)]

In [5]:
twitter_25_model.most_similar('crazy')[0][0]

'mad'

## Plan

##### 1- Traduction from french to english
##### 2- Replacing a few words from the review with most similar one (using gensim most_similar)
##### 3- Traduction from english to french
##### 4- Adding data with the new review to our dataset

Choosing a test review randomly

In [77]:
test_row = train.sample(n = 1)
test_original = str(test_row['avis'].values)
test_note = int(test_row['note'].values)
test_original

'["très déçue.\\r\\nmon premier avis a été rejeté alors je le rédige en termes qui peuvent plaire. lorsque l\'on contacte l\'assurance pour demander des renseignements on en parle jamais des franchises appliquées : -15 € pour toute consultation de moins de 75 € - 0€ remboursement  pour tout médicament de moins de 15€. c\'est parfait ! c\'est très honnête ! je l\'aurai su je n\'aurais pas pris cette assurance. c\'est pas grave, on résiliera !"]'

In [78]:
test_original = test_original.replace('\\n', '')
test_original = test_original.replace('\\r', '')
test_original

'["très déçue.mon premier avis a été rejeté alors je le rédige en termes qui peuvent plaire. lorsque l\'on contacte l\'assurance pour demander des renseignements on en parle jamais des franchises appliquées : -15 € pour toute consultation de moins de 75 € - 0€ remboursement  pour tout médicament de moins de 15€. c\'est parfait ! c\'est très honnête ! je l\'aurai su je n\'aurais pas pris cette assurance. c\'est pas grave, on résiliera !"]'

----------------------------
##### 1- Traduction from french to english

In [79]:
from googletrans import Translator
translator = Translator()

def translation_dest_src(review: str, source: str = 'fr', destination: str = 'en') -> str:
    return translator.translate(review, src = source, dest = destination).text

In [80]:
test_en = translation_dest_src(test_original,
                source = 'fr',
                destination = 'en')
test_en

'["Very disappointed. My first opinion has been rejected then I draft it in terms that can please. When contacting insurance to ask for information we never talk about applied franchises: -15 € for any consultation of less than75 € - 0 € Refund for any medicine of less than 15 €. It is perfect! It is very honest! I will have known I would not have taken this insurance. It does not matter, we will cancel! "]'

--------------------------------
##### 2- Replacing a few words from the review with most similar one (using gensim most_similar)

The idea is to pick random words in our review and replace them with similar others. We will try with a different ratio replacement to see how it reacts.

In [81]:
from random import choice

def get_most_similar_word(word: str) -> str:
    # Trying to get the most similar word
    try:
        return twitter_25_model.most_similar(word.lower())[0][0]
    # If not returning the original word
    except:
        print("Couldn't find the most similar word.")
        return word

def word_replacement(review_eng: str, replacement_ratio: float = 0.5) -> str:
    # Splitting the review by words and counting the number of words
    review_eng_words = review_eng.split()
    nb_words = len(review_eng_words)

    # Counting the number of words to replace
    nb_words_to_change = int(nb_words*replacement_ratio)

    # Creating a list with the placement of changed words, so we don't change twice the same word (could result as getting back to original)
    word_placement_changed = list()
    for words in range(nb_words_to_change):
        # Getting a random int as the word to change
        word_placement = choice([_ for _ in range(nb_words) if _ not in word_placement_changed])

        # Replacing the original word with the most similar one
        review_eng_words[word_placement] = get_most_similar_word(review_eng_words[word_placement])

        # Appending the word placement in our list to avoid changing twice the same
        word_placement_changed.append(word_placement)
        # print(' '.join(review_eng_words))

    return (' '.join(review_eng_words))

In [72]:
test_replaced = word_replacement(test_en, 0.5)
test_replaced = test_replaced.replace('_', ' ')
test_replaced

Couldn't find the most similar word.
Couldn't find the most similar word.
Couldn't find the most similar word.
Couldn't find the most similar word.
Couldn't find the most similar word.
Couldn't find the most similar word.
Couldn't find the most similar word.
Couldn't find the most similar word.
Couldn't find the most similar word.
Couldn't find the most similar word.
Couldn't find the most similar word.
Couldn't find the most similar word.


'["Prices are attractive, and easy and quick subscription. But before it gets complicated: 3 months I\'m attempting to changes bank coordinates, now do taken intothe accounts ..The email response come within An very short time (10 weeks forthe in this end Do do changes the ribs properly!) though the endorsement were complicated: ive altered home, and at first ive recieve this amendment to the wrong date of effect, when this dates is correct butThere was An error inthe my names .. Brief, I know it as soon as my contracts has 1 year ago, I change insurer! "]'

--------------------------------
##### 3- Traduction from english to french

In [73]:
test_replaced = translation_dest_src(test_replaced, source= 'en', destination= 'fr')[2:-2]
test_replaced

"Les prix sont attrayants et un abonnement facile et rapide. Mais avant qu'il ne soit compliqué: 3 mois, je tente de modifier les coordonnées bancaires, faites maintenant des comptes de la banque. La réponse par e-mail vient dans un délai très court (10 semaines surCette fin change les nervures correctement!) Bien que l'endossement ait été compliquée: ive altéré à la maison et, au premier abord, recevez cet amendement à la mauvaise date d'effet, lorsque ces dates sont correctes, il y avait une erreur dans mes noms ..Je le sais dès que mes contrats y ont 1 an, je change d'assureur! "

In [74]:
test_original

'["les prix sont attractifs, et la souscription facile et rapide.Mais après ca se complique : 2 mois que j\'essaie de changer de coordonnées bancaires, toujours pas prise en compte ..Les réponses aux mails parviennent dans un délai extrêmement long ( 10 jours pour au final ne pas changer le rib correctement!)Même les avenants sont compliqués : j\'ai changé de domicile, et dans un premier temps je reçois l\'avenant à la mauvaise date d\'effet, puis la date est correcte mais il y a une erreur dans mon nom ..!Bref, je pense que dès que mon contrat aura 1 an révolu, je change d\'assureur!"]'

--------------------------------
##### 4- Adding data with the new review to our dataset

In [75]:
def add_row(review: str, review_note: int):
    train.loc[len(train.index)] = ['data augmentation', review_note, 'data augmentation', review, None, None]

In [76]:
add_row(test_replaced, test_note)
train

,date,note,auteur,avis,assureur,produit
0,06 septem...,5,brahim--k-131532,"Meilleurs assurances, prix, solutions, écoute,...",Direct Assurance,auto
1,03 mai 20...,4,bernard-g-112497,"je suis globalement satisfait , sauf que vous ...",Direct Assurance,auto
2,21 mars 2...,5,virginie-t-107352,Prix tres abordable plusieurs options s'offren...,Direct Assurance,auto
3,10 juin 2...,4,boulain-f-116580,"je satisfait du service, une réponse très rapi...",L'olivier Assurance,auto
4,29 janvie...,1,ouaille31-51798,"Client depuis plus de 25 ans, très déçu de cet...",Matmut,auto
...,...,...,...,...,...,...
24105,data augmentation,4,data augmentation,['Je remercie Emeline qui savait quoi me donne...,None,None
24106,data augmentation,4,data augmentation,JE Remerciie Emeline Qui Saviter Quoi Me Donne...,None,None
24107,data augmentation,4,data augmentation,Toujours trop coûteux des contributions à mon ...,None,None
24108,data augmentation,4,data augmentation,Je suis insatisfait que ce prix nous convient ...,None,None
